# Paperspace Gradient: TensorFlow 2 Quick Start for Advanced Users
Last modified: Nov 18th 2021

## Purpose and intended audience

This Quick Start tutorial demonstrates TensorFlow 2 usage in a Gradient Notebook. It is aimed at users who are already familiar with TensorFlow and Python programming.

For newer users who are less familiar with TensorFlow and want a simpler overview of a model being built on Gradient, see the Quick Start for Beginners notebook, `quick_start_beginner.ipynb`.

We use [Keras](https://www.tensorflow.org/guide/keras/overview) within TensorFlow to:

- Build a neural network that classifies images
- Train this neural network
- And, finally, evaluate the accuracy of the model

followed by some next steps that you can take to proceed with using Gradient.

The material is based on the original [TensorFlow 2 Advanced Quick Start](https://www.tensorflow.org/tutorials/quickstart/advanced).

See the end of the notebook for the original copyright notice and license.

## Check that you are on a GPU instance

The notebook is designed to run on a Gradient GPU instance (as opposed to a CPU-only instance). The instance type, e.g., A4000, can be seen by clicking on the instance icon on the left-hand navigation bar in the Gradient Notebook interface. It will say if it is CPU or GPU.

![quick_start_images/example_instance_type.png](quick_start_images/example_instance_type.png)

If the instance type is CPU, you can change it by clicking *Stop Instance*, then the instance type displayed to get a drop-down list. Select a GPU instance and start up the Notebook again.

For help with instances, see the Gradient documentation on [instance types](https://docs.paperspace.com/gradient/more/instance-types) or [starting a Gradient Notebook](https://docs.paperspace.com/gradient/explore-train-deploy/notebooks).

## Setup and GPUs

Gradient avoids setup issues present on a more general system: the original Quick Start says

*Download and install TensorFlow 2. Import TensorFlow into your program: Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.*

whereas here we can immediately import TensorFlow and begin work.

In [ ]:
import tensorflow as tf

We will use the general TensorFlow subclassing interface followed by the GradientTape formulation of training, so some Keras functionality can be imported for shorthand.

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Provided that you have started this Notebook on a [Gradient GPU instance](https://docs.paperspace.com/gradient/more/instance-types), Gradient will see the GPU(s) and TensorFlow will utilize it/them.

We can list the available GPU(s) to check that TensorFlow is seeing them.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
gpus

2021-11-18 22:27:21.390370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 22:27:21.399142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 22:27:21.399777: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

By default, TensorFlow allocates all the memory on a GPU to the model being run. This is fine, until, for example, you run a container like this one that has more than one `.ipynb` notebook on it. Then the second notebook's model may fail due to the GPU being out of memory.

We can help with this by setting the memory used to [grow as needed](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth) rather than being allocated all at once at the start. This doesn't *solve* the underlying issue, but mitigates it such that the notebooks can be run several times before memory gets low.

A better solution is to allocate N logical GPUs for N notebooks, which we may add here.

In [ ]:
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2021-11-18 22:27:24.085165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 22:27:24.085853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 22:27:24.086409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 22:27:24.557492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 22:27:24.558077: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read f

## Load and prepare the data

The model shows multi-class classification of the handwritten digits 0-9 in the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

Use `tf.data` to batch and shuffle the dataset.

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Build and train the model

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing).

In [ ]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

Choose an optimizer and loss function for training.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model.

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout)
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

Test the model.

In [ ]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout)
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

2021-11-18 22:27:40.772528: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-18 22:27:41.797311: I tensorflow/stream_executor/cuda/cuda_dnn.cc:381] Loaded cuDNN version 8204
2021-11-18 22:27:43.890114: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch 1, Loss: 0.13713790476322174, Accuracy: 95.93333435058594, Test Loss: 0.05723337084054947, Test Accuracy: 98.0199966430664
Epoch 2, Loss: 0.041809797286987305, Accuracy: 98.67832946777344, Test Loss: 0.056060198694467545, Test Accuracy: 98.15999603271484
Epoch 3, Loss: 0.021004321053624153, Accuracy: 99.34500122070312, Test Loss: 0.05397671088576317, Test Accuracy: 98.36000061035156
Epoch 4, Loss: 0.012828966602683067, Accuracy: 99.5566635131836, Test Loss: 0.05806920304894447, Test Accuracy: 98.29000091552734
Epoch 5, Loss: 0.009389513172209263, Accuracy: 99.67832946777344, Test Loss: 0.06902837753295898, Test Accuracy: 98.15999603271484


The image classifier is now trained to ~99% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials).

## Next steps

To proceed with TensorFlow 2 in Gradient, you can:
    
 - Try out the quick_start_beginner.ipynb notebook in this same container if you want to see a simpler example
 - Look at other Gradient material, such as the [tutorials](https://docs.paperspace.com/gradient/get-started/tutorials-list), [ML Showcase](https://ml-showcase.paperspace.com), [blog](https://blog.paperspace.com), or [community](https://community.paperspace.com)
 - Start writing your own projects, using our [documentation](https://docs.paperspace.com/gradient) when needed
 
If you get stuck or need help, [contact support](https://support.paperspace.com), and we will be happy to assist.

Good luck!

## Original TensorFlow copyright notice and license
##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# # https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.